# Softmax Word2Vec method

> About the autor: Jinming Yang is currently an undergraduate school student at Sun Yet-san University who focuses on transportation research and machine learning.

For a 10000 word vocabulary, using one-hot vectors as the representation of each word in a neural network is computational expensive. Besides, one-hot vectors can not represent the correlations between words like "Soviet" and "Union", "United" and "States"... So we need a new representation of those words. These new representations of words have much lower dimension like 128. The correlations between strong connected words can be reflected in their corresponding representations.

In order to do that, **Word2Vec** was introduced to derive those word representations. Hereby, we'll mainly focuse on the **Softmax Word2Vec method**.

## Methodology
Basically the Softmax Word2Vec method use an three layer autoencoder neural network to derive word representations. The first layer is the input layer consists of 10000 neurons(units). The second layer is the hidden layer which consists of 128 neurons(no activation function). The third layer is the softmax layer which consists of 10000 neurons.

**Network setting**

|`Network Layer`  |`Number of neurons`|`Activation`|
|:----------------|------------------:|-----------:|
|**Input layer**  |     **10000**     |  **None**  |
|**Hidden layer** |      **128**      |  **None**  |
|**Output layer** |     **10000**     |**Sigmoid** |

In [3]:
import numpy as np
import tensorflow as tf
import collections
import math

### Preparing the corpus
Downloading the corpus in "mattmahoney.net/dc/text8.zip". For researchers in main land China the website won't be available. You can also download it in my GitHub repositry "Learning Machine Learning".

In [6]:
#Read the file
f = open("ptbdata/ptb.train.txt","r")
rawData = f.read()
print(rawData[:1000])

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 
 the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said 
 <unk> inc. the unit of new york-based <unk> corp. that makes kent cigarettes stopped using <unk> in its <unk> cigarette filters in N 
 although preliminary findings wer

After opening the file, we can transfer the content in the file to string format. Then we can split the text string to individual words by blank.

In [7]:
# Transfer the raw data as strings using Tensorflow. 
# Then split it into individual words
dataStr = tf.compat.as_str(rawData) #Convert to string
print('Split:')
data = dataStr.split() #Split by blank
print(data[:100])
print(len(data))

Split:
['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of', 'workers', 'exposed', 'to', 'it']
887521


### Create dictionary and numerical representation of the corpus
We have the words sequence in the corpus. Now we want to use numbers to represent each word and create the dictionary of words and their corresponding number. After that, we can convert the whole text from word sequence to number sequence.

#### Count word number
First, count the number of occurence of each word in the corpus.

In [4]:
counted_words = collections.Counter(data)  #counted_words is a dictionary {'word1': num_1, 'word2': num_2, ...}
print('%d different words were found in the corpus'%(len(counted_words)))
print()
print('Part of the dictionary:')
dict(list(counted_words.items())[0:10])    #print 10 of them

253854 different words were found in the corpus

Part of the dictionary:


{'anarchism': 303,
 'originated': 572,
 'as': 131815,
 'a': 325873,
 'term': 7219,
 'of': 593677,
 'abuse': 563,
 'first': 28810,
 'used': 22737,
 'against': 8432}

#### Select top frequent words
From above, we can see that there are 253854 diffenent words in the corpus. However most of them have very few occurence, building a dictionary for them is highly uneconomical and inefficient. So we just choose the top 9999 frequent words to build the dictionary. The rest of the words which are less likely to occur were classified as 'LFW' aka 'Low Frequency Words'.

In [5]:
freq_counted_words = dict(counted_words.most_common(9999))
print('There are %d words in the selected dictionary'%(len(freq_counted_words)))

There are 9999 words in the selected dictionary


Now conbine all other less frequently occured words as 'LFW', count their number of occurence.

In [7]:
lfw_count = 0
for word in counted_words:
    if not (word in freq_counted_words.keys()):
        lfw_count += 1
word_count_dict = {'lfw': lfw_count}
word_count_dict.update(freq_counted_words)
print('There are %d words in the dictionary'%(len(word_count_dict)))
print()
print('The first 5 entries in the dictionary is:')
dict(list(word_count_dict.items())[:5])

There are 10000 words in the dictionary

The first 5 entries in the dictionary is:


{'lfw': 243855, 'the': 1061396, 'of': 593677, 'and': 416629, 'one': 411764}

We have already selected the top 10000 frequent words in the corpus. Now we need to index them with numbers aka establish the word2number projection.

In [8]:
ind = 0
word_dict = {}
for word in word_count_dict:
    word_dict.update({word: ind})
    ind += 1
print('The first 5 entries in the dictionary "word_dict" is: ')
dict(list(word_dict.items())[:5])

The first 5 entries in the dictionary "word_dict" is: 


{'lfw': 0, 'the': 1, 'of': 2, 'and': 3, 'one': 4}

Since we have already built the dictionary of words and their corresponding number(index). We can now transfer the initial corpus to a number sequence.

In [9]:
num_corpus = []
for word in data:
    if word in word_dict.keys():
        num_corpus.append(word_dict[word])
    else:
        num_corpus.append(word_dict['lfw'])
print('The first 100 words in the corpus represented by their corresponding indeces are:')
print(num_corpus[:100])

The first 100 words in the corpus represented by their corresponding indeces are:
[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 0, 134, 1, 0, 2, 1, 103, 855, 3, 1, 0, 0, 2, 1, 151, 855, 3581, 1, 195, 11, 191, 59, 5, 6, 0, 215, 7, 1325, 105, 455, 20, 59, 2732, 363, 7, 3673, 1, 709, 2, 372, 27, 41, 37, 54, 540, 98, 12, 6, 1424, 2758, 19, 568, 687, 7089, 1, 248, 5234, 11, 1053, 28, 1, 321, 249, 0, 2878, 793, 187, 5234, 12, 6, 201, 603, 11, 1, 1135, 20, 2622, 26, 8984, 3, 280, 32, 4148, 142, 60, 26, 6438]


We have the dictionary to project words to indeces. We now have to build a dictionary to project each index to its corresponding word. So that we can recover text from index sequence.

In [10]:
index_dict = dict(zip(word_dict.values(), word_dict.keys()))
print('The first 10 entries in dictionary index_dict is: ')
dict(list(index_dict.items())[:10])

The first 10 entries in dictionary index_dict is: 


{0: 'lfw',
 1: 'the',
 2: 'of',
 3: 'and',
 4: 'one',
 5: 'in',
 6: 'a',
 7: 'to',
 8: 'zero',
 9: 'nine'}

## Skip-gram

The **skip-gram** conscept is used to create batches for the autoencoder neural network. It basically has two main conscepts:

* **gram**: **gram** is a fix-sized sliding window over the corpus.

* **skip**: **skip** is the number of times a word repeated in the dataset with different context words.

The central word of the gram is called the **target**. And the rest of the words in the gram is the context words of the **target**. Utilising every context word may be computational expensive, thus we just randomly choose **skip** different context words for one **target** to train the autoencoder neural network.

### Neural network inputs and their labels

Based on the conscept above, the input of the neural network is the initial representation of each **target** word. For example, in this experiment setting, the input would be a 10000 dimension one-hot vector denoting each **target**. And the label would also be a 10000 dimension one-hot vector denoting one of the context word of that  **target**. Basically, the inputs and their label are this kind of combination: 

${[target_1, context_1^{(i)}], [target_1, context_2^{(i)}], ..., [target_1, context_{skip}^{(i)}], ..., [target_{ng}, context_1^{(i)}], [target_{ng}, context_2^{(i)}], [target_{ng}, context_{skip}^{(i)}]}$.

<img src="images/skip_gram.png" alt="Drawing" style="width: 500px;"/>

The following function can generate a mini-batch with size **batch_size**. **$target$** and **context** were generated separatley, both of which have the length of **batch_size**.

In [35]:
data_index = 0
# generate batch data
def generate_batch(data, batch_size, skip, sub_gram):
    global data_index
    assert batch_size % skip == 0
    assert skip <= 2 * sub_gram
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * sub_gram + 1  # [ sub_gram input_word sub_gram]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // skip):
        target = sub_gram  # input word at the center of the buffer
        targets_to_avoid = [sub_gram]
        for j in range(skip):
            while target in targets_to_avoid:
                target = np.random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * skip + j] = buffer[sub_gram]  # this is the input word
            context[i * skip + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

## Autoencoder Network Setting
Neural network settings:

In [27]:
vocabulary_size = 10000
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector aka the number of units in the hidden layer.
sub_gram = 2          # (Span_of_gram-1)/2
skip = 2              # How many times to reuse an input to generate a context.

Set placeholders for inputs and outputs:

In [28]:
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])

Set variables in the network: the embedding here is a $(10000\times128)$ weight matrix

In [29]:
# weight matrix between input layer and hidden layer
embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

# get the corresponding embedding of input word.
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

# weight matrix between hidden layer and the softmax layer
weights = tf.Variable(tf.truncated_normal([embedding_size, vocabulary_size],
                          stddev=1.0 / math.sqrt(embedding_size)))

# biases of softmax layer
biases = tf.Variable(tf.zeros([vocabulary_size]))

hidden_out = tf.matmul(embed, weights) + biases

Convert the context into a one-hot format:

In [30]:
train_one_hot = tf.one_hot(train_context, vocabulary_size)

Using crossentropy as the loss function.

In [31]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, 
    labels=train_one_hot))

Set a gradient descent minimizer for the **cross_entropy**.

In [32]:
# Construct the SGD optimizer using a learning rate of 1.0.
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)

Initialize all the parameters.

In [33]:
init = tf.global_variables_initializer()

### Neural Network training

In [39]:
num_steps = 100000
with tf.Session() as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_context = generate_batch(num_corpus,
        batch_size, skip, sub_gram)
    feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

Initialized
Average loss at step  0 :  9.340882301330566
Average loss at step  2000 :  6.716523780465126
Average loss at step  4000 :  6.247654334843158
Average loss at step  6000 :  6.252160863399506
Average loss at step  8000 :  6.177210550308228
Average loss at step  10000 :  5.95334085381031
Average loss at step  12000 :  6.044154072523117
Average loss at step  14000 :  5.928458213567734
Average loss at step  16000 :  5.717213647961617
Average loss at step  18000 :  5.702529030680656
Average loss at step  20000 :  6.0616026866436
Average loss at step  22000 :  6.039542630553245
Average loss at step  24000 :  5.96240766775608
Average loss at step  26000 :  5.943196316123009
Average loss at step  28000 :  5.98998781311512
Average loss at step  30000 :  5.848000244021415
Average loss at step  32000 :  6.081995312213897
Average loss at step  34000 :  6.031385659337044


KeyboardInterrupt: 